<a href="https://colab.research.google.com/github/OscarRojasG/Experimentos-GPTValidator/blob/main/Framework_Experimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Framework para experimentos GPTValidator

El objetivo de este framework consiste en proporcionar un conjunto de funciones previamente implementadas para facilitar la evaluación y comparación de prompts para el proyecto EvaluAI.

## Instrucciones de uso

### 1. Cargar dataset

1. Subir archivo .xlsx con todos los datasets combinados. Debe contener al menos 5 columnas con los siguientes datos:
    * **Contexto** (context): Conocimiento previo que necesita el modelo para evaluar la respuesta del estudiante.
    * **Pregunta** (question)
    * **Respuesta** (answer): Respuesta del estudiante
    * **Evaluación manual** (real_eval): Puntaje de referencia dado por uno o más evaluadores humanos.
    * **Dataset de origen** (dataset): Dataset del cual provienen los datos para una fila en particular.

2. Especificar el nombre de las columnas en un diccionario con la siguiente estructura (ejemplo):

    ```
    column_data = {
            "context": "Contexto",
            "question": "Pregunta",
            "answer": "Respuesta",
            "real_eval": "EvalManual",
            "dataset": "Dataset"
    }
    ```

3. Cargar dataset con la función `load_dataset(path, sheet_name, column_data)`

    - **path** - Ruta del archivo xlsx a utilizar.
    - **sheet_name** - Nombre de la hoja donde se encuentran los datos.
    - **column_data** - Diccionario explicado en el punto anterior.

### 2. Definir miniprompts a evaluar

1. Subir carpeta con la colección de miniprompts. Los miniprompts deben estar definidos en archivos txt y agrupados en carpetas por categoría (ejemplos, analysis, feedback, etc.). **Se pueden crear nuevas carpetas sin restricciones**.

2. Seleccionar los miniprompts a utilizar para la generación de los prompts. Estos deben ser definidos en un diccionario donde la clave corresponde al nombre de la carpeta y el valor es el nombre del archivo txt con el miniprompt correspondiente. Se puede utilizar el comodín * como valor para probar con todos los miniprompts de esa carpeta.

    ```
    prompt_data = {
        "examples": "examples_XX.txt",
        "context": "knowledge_XX.txt",
        "question": "...",
        "answer": "...",
        "instructions": {
            "analysis": "*",
            "feedback": "...",
            "score": "...",
        }
    }
    ```

3. Cargar prompts con la función ``generate_prompts(prompt_data, prompt_folder)``

    - **prompt_data** - Diccionario explicado en el punto anterior.
    - **prompt_folder** - Ruta de la carpeta donde se encuentra la colección de miniprompts. (Ej: Experiments/Miniprompts)

#### Notas adicionales

En caso de tener problemas al subir la carpeta a colab, subirla como zip y ejecutar el comando

```
!unzip nombre_carpeta.zip
```

Los criterios deben ser definidos en las primeras líneas del miniprompt score con el símbolo $.

```
$correctness
$completeness
$clarity
(score) Assign a score between 0 and 10 to different criteria based on the student's answer and the generated feedback. Criteria are: correctness, completeness, clarity.
```

Las instrucciones de salida de algunos miniprompts como feedback y analysis, se pueden definir usando el símbolo # en la primera línea del archivo.

```
#very detailed feedback considering previous analysis (in spanish, within 150 words)
(feedback)
Provide Feedback to the student considering the analysis. **Do not be strict at all**.
It is enough that the student answer correctly and more or less completely the question. **Do not ask for additional information**.
Start by stating whether the answer is good/excelent or poor/insatisfactory.
If the answer is good/excelent, affirm the student's understanding and potentially add a brief note on why their response was particularly effective or comprehensive.
If the answer is poor/insatisfactory, clearly identify the inaccuracies or errors. Provide specific suggestions on how to improve, ensuring the feedback is constructive.
Within 150 words. In Spanish.
```

### 3. Realizar experimento

Evaluar los prompts con la función ``experiment(dataset, prompts, repetitions, eval_function, eval_params=None, train_set_size=60, test_set_size=100, seed=42)``

- **dataset** - Objeto de tipo DataFrame cargado en el punto 1.
- **prompts** - Colección de prompts cargados en el punto 2.
- **repetitions** - Número de repeticiones que se desea realizar la evaluación de cada prompt. Cada evaluación implica un nuevo conjunto de entrenamiento/prueba.
- **eval_function** - Método para convertir los puntajes GPT a escala 0-3. Puede ser "cuts" o "map".
- **eval_params** - Lista de parámetros usados para la conversión de puntajes. Son distintos dependiendo del valor de *eval_function*.
- **train_set_size** - Tamaño total $n$ del conjunto de entrenamiento. El conjunto de entrenamiento se encuentra compuesto por $n/4$ muestras de cada puntaje (balanceado).
- **test_set_size** - Tamaño total del conjunto de prueba (no balanceado).
- **seed** - Semilla utilizada para la creación de los conjuntos de prueba y entrenamiento.

Para ``eval_function = "cuts"`` los parámetros siguen la estructura $[w_1, w_2, ..., w_m, a, b, c]$

Para ``eval_function = "map"`` los parámetros siguen la estructura $[w_1, w_2, ..., w_m, a, b]$

En ambos casos, los primeros $m$ parámetros corresponden a las ponderaciones de cada criterio (en el mismo orden en que fueron definidos dentro del miniprompt score). La suma de las ponderaciones debe ser 1. En caso de no haber criterios, existe una única ponderación igual a 1.

Para la evaluación por cortes, los parámetros $a, b, c$ corresponden a los puntajes de corte ordenados de menor a mayor. Por ejemplo, para un prompt con 2 criterios, una lista de parámetros válidos podría ser $[0.7, 0.3, 3, 5, 7]$

Para la evaluación por mapeo, los parámetros $a, b$ representan los puntajes en escala 0-10 que corresponden a los puntajes 1 y 2 en escala 0-3. Por ejemplo:

$\text{map}(0) = 0$

$\text{map}(a) = 1$

$\text{map}(b) = 2$

$\text{map}(10) = 3$

# Implementación paso a paso

In [ ]:
!pip install openai==0.28 &> /dev/null
!pip install openai-multi-client &> /dev/null
!git clone https://github.com/rilianx/GPTEvaluator &> /dev/null

## Cargar dataset

In [ ]:
import pandas as pd
from IPython.display import display

# Muestra información relevante del dataset
def show_dataset_info(dataset):
    display(dataset.head())
    print()
    print(dataset.value_counts("real_eval"), end="\n\n")
    print(dataset.value_counts("dataset"))
    pass

# Carga un dataset a partir de un archivo xlsx y valida sus columnas
def load_dataset(path, sheet_name, column_data):
    df = pd.read_excel(path, sheet_name=sheet_name)

    mandatory_cols = ["context", "question", "answer", "real_eval", "dataset"]
    for key in mandatory_cols:
        if key not in column_data.keys():
            raise Exception(f"Error: Debe especificar la columna para la variable {key}")

        value = column_data[key]
        if value not in df.columns:
            raise Exception(f"Error: La columna {value} no existe")

        df = df.rename(columns={value: key})

    df = df[mandatory_cols]
    show_dataset_info(df)
    return df

In [ ]:
column_data = {
    "context": "Contexto",
    "question": "Pregunta",
    "answer": "Respuesta",
    "real_eval": "Promedio Redondeado",
    "dataset": "DatasetProveniente"
}

df = load_dataset("datasets_v2.xlsx", "AllDatasets (1dif)", column_data)

,context,question,answer,real_eval,dataset
0,"Un montículo binario se mantiene completo insertando elementos en el siguiente espacio libre de la última fila y ""flotando"" el elemento si es necesario. Para eliminar, se intercambia el elemento de la raíz con el último elemento y luego se elimina el último elemento. Esta inserción y eliminación controlada asegura que el montículo siempre sea un árbol binario completo.",¿Cómo se asegura que un montículo binario mantenga su forma completa (perfectamente balanceado) al agregar o eliminar elementos?,"Para asegurar que se mantenga balanceado se comparan los nodos con sus nodos hijos, de modo que el valor máximo o mínimo siempre se mantengan en lo mas arriba del árbol, en la altura 0, y los nodos hijos se acomoden en base al nodo origen",0,C3-Sample100
1,"La utilidad de tener nodos con múltiples claves en árboles B/B+ radica en la capacidad de almacenar más información en cada nodo, lo que reduce la altura del árbol y, por lo tanto, el número de accesos a disco necesarios para buscar, insertar o eliminar datos.",¿Cuál es e la utilidad de tener nodos con múltiples claves en árboles B/B+? Explique.,Es útil ya que gracias a eso la altura de esos arboles es significativamente mas baja en comparación a la de otros arboles binarios de búsqueda. Además que es mas eficiente en la búsqueda de datos en específicos y en rango.,2,C3-Sample100
2,"La utilidad de tener nodos con múltiples claves en árboles B/B+ radica en la capacidad de almacenar más información en cada nodo, lo que reduce la altura del árbol y, por lo tanto, el número de accesos a disco necesarios para buscar, insertar o eliminar datos.",¿Cuál es e la utilidad de tener nodos con múltiples claves en árboles B/B+? Explique.,"El tener nodos con múltiples claves en árboles B/B+ garantiza una mayor eficiencia, pues al ser árboles autobalanceables (gracias a sus divisiones y fusiones) asegura que su altura se mantendrá mínima, lo cual hace que sus operaciones de búsqueda, inserción y eliminación sean menos costosas lo que implica una complejidad temporal de O(log n).",2,C3-Sample100
3,"Los árboles binarios autobalanceables como AVL y rojo-negro realizan rotaciones y ajustes durante las operaciones de inserción y eliminación para mantener el árbol balanceado. Estas rotaciones y rebalanceos garantizan que la altura del árbol se mantenga O(log n), lo que asegura que las operaciones de búsqueda, inserción y eliminación tengan una complejidad de tiempo logarítmica.",¿Por qué las operaciones de los árboles binarios autobalanceables (AVL y rojo-negro) tienen complejidad de tiempo logarítmica?,"Los árboles AVL y rojo-negro tienen complejidad de tiempo logarítmica debido a que estos árboles se balancean automáticamente después de cada inserción o eliminación, usando el método de rotaciones que mantienen el orden y equilibrio en el árbol. Con el balanceo se asegura que la altura del árbol sea baja y cada nivel este lo más lleno posible, lo que mantiene la complejidad de O(log n) en las operaciones.",2,C3-Sample100
4,"Un problema puede resolverse utilizando grafos y algoritmos de búsqueda si puede representarse mediante nodos (objetos, estados) y aristas (relaciones, transiciones) y si se busca una conexión, ruta, o patrón entre estos nodos. Para modelar el problema: 1) Identificar los nodos y aristas. 2) Determinar la estructura del grafo (dirigido/no dirigido, ponderado/no ponderado). 3) Elegir un algoritmo de búsqueda (BFS, DFS, Dijkstra, etc.) basado en el tipo de problema. Para resolverlo, aplicar el algoritmo seleccionado al grafo modelado, siguiendo los pasos específicos del algoritmo para encontrar la solución deseada.",¿Qué características debería tener un problema para poder resolverlo utilizando grafos y algoritmos de búsqueda? ¿Qué pasos debo seguir para modelar el problema? ¿Qué hago para resolverlo?,"Un problema para poder ser resulto con grafos y algoritmos de búsqueda debe plantearse un estado inicial y un estado final esperado por 


real_eval
3    127
2     75
0     52
1     36
Name: count, dtype: int64

dataset
C2-Nan                  92
C3-Sample100            91
C2-Sample100            90
C1-OscarBadAnswers20    17
Name: count, dtype: int64


## Generación prompts

In [ ]:
import pprint
import copy
import json
import os
import re

class Prompt():
    def __init__(self, structure, instructions, base_folder):
        self.structure = structure
        self.instructions = instructions
        self.base_folder = base_folder
        self.raw_text_structure = None
        self.text_structure = None
        self.criteria = None
        self.output_instructions = None
        self.prompt = None

        self.read_files()
        self.extract_metadata()
        self.build_prompt()

    # Retorna la estructura base del prompt (diccionario)
    def base_structure(self):
        structure = copy.deepcopy(self.structure)
        structure['instructions'] = {}
        for i in self.instructions:
            structure['instructions'][i] = structure[i]
            structure.pop(i, None)
        return structure

    # Crea un diccionario con el contenido de cada archivo en la estructura
    def read_files(self):
        self.raw_text_structure = copy.deepcopy(self.structure)

        for key, value in self.raw_text_structure.items():
            if key == "instructions": continue

            path = f"{self.base_folder}/{key}/{value}"
            try:
                self.raw_text_structure[key] = open(path, 'r', encoding='utf-8').read()
                self.raw_text_structure[key] += "\n\n"
            except:
                raise Exception(f"Error: El archivo {path} no existe")

    # Extrae metadatos de los archivos como los criterios e instrucciones de salida
    def extract_metadata(self):
        self.text_structure = copy.deepcopy(self.raw_text_structure)

        if 'score' in self.text_structure:
            lines = self.text_structure['score'].split('\n')
            self.criteria = [line[1:] for line in lines if line.startswith('$')]
            text = [line for line in lines if not line.startswith('$')]
            self.text_structure['score'] = '\n'.join(text)

        self.output_instructions = {}
        for key, value in self.text_structure.items():
            if value.startswith('#'):
                m = re.search(r'#(.*?)\n', value).group(1)
                self.output_instructions[key] = m
                self.text_structure[key] = '\n'.join(value.split('\n')[1:])

    # Construye el prompt en formato string
    def build_prompt(self):
        self.prompt = ""
        for key, value in self.text_structure.items():
            self.prompt += value

        output = "I expect a dict in python as answer: {{"
        for key, value in self.output_instructions.items():
            output += f'"{key}": \'{value}\', '

        if len(self.criteria) > 0:
            for c in self.criteria:
                output += f'"{c}": {c}_score, '
            output = output[:-2]
        else:
            output += '"score": score'

        output += "}}\n\nPython dict:"
        self.prompt += output


# Procesa y elimina los diccionarios anidados de prompt_data
def normalize_prompt_dict(prompt_data):
    instructions = []
    if "instructions" in prompt_data:
        for (key, value) in prompt_data["instructions"].items():
            prompt_data[key] = value
            instructions.append(key)

    prompt_data["instructions"] = "Instructions:\n"
    return prompt_data, instructions

# Retorna la lista de archivos para reemplazar el comodín *
def expand_prompt_data(prompt_data, prompt_folder):
    wildcard_field = None
    for key, value in prompt_data.items():
        if value == "*":
            wildcard_field = key
            break

    if not wildcard_field: return None, None

    wildcard_files = []
    path = f"{prompt_folder}/{wildcard_field}"
    for file in sorted(os.listdir(path)):
        if os.path.isfile(os.path.join(path, file)):
            wildcard_files.append(file)

    return wildcard_field, wildcard_files

# Genera una lista con los prompts a evaluar
def generate_prompts(prompt_data, prompt_folder):
    template, instructions = normalize_prompt_dict(prompt_data)
    wildcard_field, wildcard_files = expand_prompt_data(template, prompt_folder)
    prompts = []

    if wildcard_field == None:
        prompt = Prompt(template, instructions, prompt_folder)
        prompts.append(prompt)
        print(prompt.prompt)
        return prompts

    for file in wildcard_files:
        structure = copy.deepcopy(template)
        structure[wildcard_field] = file
        prompt = Prompt(structure, instructions, prompt_folder)
        prompts.append(prompt)

    # Visualizar
    template = copy.deepcopy(prompts[0])
    template.raw_text_structure[wildcard_field] = f"{{{wildcard_field}}}\n\n"
    template.extract_metadata()
    template.build_prompt()
    print(template.prompt)

    print(f"\n\nArchivos a utilizar ({len(wildcard_files)}):\n")
    print("\n".join(wildcard_files))

    return prompts

In [ ]:
prompt_data = {
    "examples": "examples_1.txt",
    "context": "knowledge_1.txt",
    "question": "question_1.txt",
    "answer": "answer_1.txt",
    "instructions": {
        "analysis": "analysis_1.txt",
        "feedback": "feedback_1.txt",
        "score": "score_1.txt",
    }
}

prompt_folder = "GPTEvaluator/Experiments/Miniprompts"

prompts = generate_prompts(prompt_data, prompt_folder)

**Examples:**
Q: ¿Cómo se podría implementar un historial de navegación web usando dos pilas? El historial debe permitir ir hacia atrás y adelante con las páginas previamente visitadas. Describa un algoritmo.
Incorrect Answer: Usamos dos pilas para ir hacia adelante y hacia atrás en el historial.  (Score: 0)

Q: ¿Cómo se busca un valor en un árbol rojo-negro? Explique el proceso paso a paso.
Incorrect Answer: PAra buscar el valor en un árbol rojo-negro debemos pasar por nodos rojos y negros hasta encontrar el valor. (Score: 0)

Q: ¿Por qué el acceso a un elemento específico en un arreglo es O(1), es decir, no depende de la cantidad de datos?
Incorrect Answer: El acceso es O(1) por que toma un tiempo constante y no depende de la cantidad de datos. (Score: 0)

Q: ¿Cuando se recomienda utilizar arreglos en vez de listas enlazadas? Haga referencia a complejidades temporales en su explicación.
Incorrect Answer: Un arreglo es recomendable en determinadas situaciones, mientras que la lista en

## Optimización parámetros

In [ ]:
from scipy.optimize import differential_evolution
import numpy as np


### Optimización de cortes y ponderaciones ###
class CutsEvaluator():
    @staticmethod
    def f(x, theta):
        score = np.dot(x, theta[:-3])
        y_pred = np.where(score > theta[-1], 3, np.where(score > theta[-2], 2, np.where(score > theta[-3], 1, 0)))
        return y_pred

class CutsOptimizer():
    def __init__(self, criteria_scores, real_scores):
        bounds =  [(0, 1) for _ in range(len(criteria_scores[0]))] + [(1, 4), (4, 7), (7, 10)]
        result = differential_evolution(self.error, bounds, args=(criteria_scores, real_scores), seed=1, strategy='rand1exp', mutation=(0,1), recombination=1)
        self.params = result.x.tolist()

    def error(self, theta, x, y):
        y_pred = CutsEvaluator.f(x, theta)
        mse = np.mean((y - y_pred) ** 2)
        penalty = 1e6 * np.abs(np.sum(theta[:-3]) - 1)
        return mse + penalty


####### Optimización mapeo de puntajes ########
class MapEvaluator():
    @staticmethod
    def inverse_map(y_pred, theta):
        a, b = theta[-2], theta[-1]
        def single_inverse_map(y):
            if y <= a:
                return y / a
            elif a < y <= b:
                return 1 + (y - a) / (b - a)
            else:
                return 2 + (y - b) / (10 - b)

        return np.array([single_inverse_map(y) for y in y_pred])

    @staticmethod
    def f(x, theta):
        y_pred = np.dot(x, theta[:-2])
        return MapEvaluator.inverse_map(y_pred, theta)

class MapOptimizer():
    def __init__(self, criteria_scores, real_scores):
        bounds =  [(0, 1) for _ in range(len(criteria_scores[0]))] + [(1, 9), (1, 9)]
        result = differential_evolution(self.error, bounds, args=(criteria_scores, real_scores), seed=1, strategy='rand1exp', mutation=(0,1), recombination=1)
        self.params = result.x.tolist()

    def error(self, theta, x, y):
        y_pred = MapEvaluator.f(x, theta)
        mse = np.sum((y - y_pred) ** 2)
        penalty = 1e6 * np.abs(np.sum(theta[:-2]) - 1)
        a, b = theta[-2], theta[-1]
        if a > b: penalty += (a - b) * 1e5
        return mse + penalty

# Convierte una lista de diccionarios en una lista de tuplas
def get_x(gpt_dicts, criteria):
    if len(criteria) > 0:
        return [
            [gpt_dict[key] for key in criteria]
            for gpt_dict in gpt_dicts
        ]

    return [[gpt_dict['score']] for gpt_dict in gpt_dicts]

# Obtiene los parámetros óptimos para disminuir el error
def optimize_params(gpt_dicts, real_scores, criteria, eval_function):
    criteria_scores = get_x(gpt_dicts, criteria)
    if eval_function == "map":
        optimizer = MapOptimizer(criteria_scores, real_scores)
    if eval_function == "cuts":
        optimizer = CutsOptimizer(criteria_scores, real_scores)

    return optimizer.params

# Convierte los puntajes GPT a puntajes normalizados (0-3)
def convert_gpt_scores(gpt_dicts, real_scores, criteria, eval_function, eval_params):
    criteria_scores = get_x(gpt_dicts, criteria)
    if eval_function == "map":
        return MapEvaluator.f(criteria_scores, eval_params)
    if eval_function == "cuts":
        return CutsEvaluator.f(criteria_scores, eval_params)

In [ ]:
gpt_dicts = [
    {
        'score': 9
    },
    {
        'score': 8
    },
    {
        'score': 4
    }
]

real_scores = [3, 2, 1]
criteria = []

eval_params = optimize_params(gpt_dicts, real_scores, criteria, "map")
print(eval_params)
convert_gpt_scores(gpt_dicts, real_scores, criteria, "map", eval_params)

[1.0, 4.000037881048441, 7.4999888512711514]


array([2.60000178, 2.20000357, 0.99999053])

In [ ]:
gpt_dicts = [
    {
        'relevance': 8,
        'clarity': 9,
        'precision': 7
    },
    {
        'relevance': 8,
        'clarity': 5,
        'precision': 3
    },
    {
        'relevance': 4,
        'clarity': 1,
        'precision': 2
    }
]

real_scores = [3, 2, 1]
criteria = ['relevance', 'clarity', 'precision']

eval_params = optimize_params(gpt_dicts, real_scores, criteria, "map")
print(eval_params)
convert_gpt_scores(gpt_dicts, real_scores, criteria, "map", eval_params)

[0.007063405579985471, 0.9736191598576271, 0.01931743420011689, 1.0250005164759606, 4.822870728613848]


array([2.79801581, 2.03084424, 1.00408311])

## Experimentos

In [ ]:
import random
from GPTEvaluator.GPTEvaluator import chat_gpt_multiple
from openai_multi_client import OpenAIMultiClient
import openai
from google.colab import userdata
from sklearn.metrics import mean_squared_error
import os
from datetime import datetime, timedelta
import json
import numpy

openai.api_key = userdata.get('api_key')

class SetPair():
    def __init__(self, train_set, test_set):
        self.train_set = train_set
        self.test_set = test_set

# Divide el dataset en conjuntos de entrenamiento/prueba
def generate_sets(dataset, repetitions, train_set_size=60, test_set_size=100, group_size=15, seed=42):
    sets = []
    random.seed(seed)

    group_size = train_set_size // 4
    for i in range(repetitions):
        train_set = dataset.groupby('real_eval', group_keys=False).apply(lambda x: x.sample(group_size, random_state=random.randint(0,100000)))
        test_set = dataset[~dataset.index.isin(train_set.index)]
        test_set = dataset.sample(n=test_set_size, random_state=random.randint(0,100000))
        sets.append(SetPair(train_set, test_set))

    return sets

# Genera las respuestas con ChatGPT
def eval_gpt(df, prompt):
    api = OpenAIMultiClient(endpoint="chats", data_template={"model": "gpt-4o-mini", "temperature": 0.1, "n": 1, "timeout":10}, concurrency=50, wait_interval=1, max_retries=3, retry_max=10, retry_multiplier=1)

    texts = []
    for i, row in df.iterrows():
        text = prompt.format(Question=row['question'], Answer=row['answer'], Context=row['context'])
        texts.append(text)

    answers_gpt = chat_gpt_multiple(api, texts)
    return answers_gpt

# Extrae diccionario de salida de las respuestas GPT
def extract_dicts(answers_gpt):
    pattern = r'\{[^{}]+\}'

    gpt_dicts = []
    for answer_gpt in answers_gpt:
        try:
            answer = re.findall(pattern, answer_gpt[0])[0]
            gpt_dicts.append(eval(answer))
        except Exception as e:
            print(f"Error al extraer diccionario. Respuesta GPT: \n{answer_gpt[0]}\n\n")
            gpt_dicts.append(None)

    return gpt_dicts

# Elimina filas del dataset donde hubo errores en la salida GPT
def clean_set(dataset, gpt_dicts):
    for i in reversed(range(len(gpt_dicts))):
        if gpt_dicts[i] is None:
            gpt_dicts.pop(i)
            dataset.drop(index=i, inplace=True)

# Obtiene los puntajes reales de un dataset
def get_real_scores(dataset):
    return dataset['real_eval'].tolist()

# Prepara el set de entrenamiento y obtiene los parámetros óptimos para disminuir el error
def train(train_set, prompt, criteria, eval_function):
    train_set = train_set.copy()
    answers_gpt = eval_gpt(train_set, prompt)
    gpt_dicts = extract_dicts(answers_gpt)
    clean_set(train_set, gpt_dicts)
    real_scores = get_real_scores(train_set)
    return optimize_params(gpt_dicts, real_scores, criteria, eval_function)

# Prepara el set de prueba y calcula las métricas del modelo preentrenado usando el conjunto de prueba
def test(test_set, prompt, criteria, eval_function, eval_params):
    test_set = test_set.copy()
    answers_gpt = eval_gpt(test_set, prompt)
    gpt_dicts = extract_dicts(answers_gpt)
    clean_set(test_set, gpt_dicts)
    real_scores = get_real_scores(test_set)
    pred_scores = convert_gpt_scores(gpt_dicts, real_scores, criteria, eval_function, eval_params)

    result_set = test_set.copy(deep=True)
    result_set['gpt_eval'] = pred_scores
    return calculate_mse(result_set)

# Retorna un dataset con el MSE por grupo
def calculate_mse(result_set):
    mse_dict = result_set.groupby('dataset').apply(lambda x: mean_squared_error(x['real_eval'], x['gpt_eval'])).to_dict()
    overall_mse = mean_squared_error(result_set['real_eval'], result_set['gpt_eval'])
    mse_dict['All'] = overall_mse
    return mse_dict

# Evalúa una lista de prompts obtienendo el MSE promedio en M repeticiones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
def experiment(dataset, prompts, repetitions, eval_function, eval_params=None, train_set_size=60, test_set_size=100, seed=42):
    sets = generate_sets(dataset, repetitions, train_set_size, test_set_size, seed)
    stats = []

    for i, prompt_data in enumerate(prompts):
        prompt = prompt_data.prompt
        criteria = prompt_data.criteria
        stats.append([])

        for j in range(repetitions):
            train_set = sets[j].train_set
            test_set = sets[j].test_set

            iter_params = eval_params
            if not eval_params:
                print(f"Entrenando Prompt {i+1} con Train Set {j+1}")
                iter_params = train(train_set, prompt, criteria, eval_function)

            print(f"\nEvaluando Prompt {i+1} con Test Set {j+1}")
            metrics = test(test_set, prompt, criteria, eval_function, iter_params)
            stats[i].append({
                "MSE": metrics,
                "params": iter_params
            })
            print()

    filename = save_stats(prompts, stats, eval_function, train_set_size, test_set_size)
    read_results(filename)

    return stats

# Guarda resultados en un archivo JSON
def save_stats(prompts, stats, eval_function, train_set_size, test_set_size):
    data = {
        "train_set_size": train_set_size,
        "test_set_size": test_set_size,
        "repetitions": len(stats[0]),
        "results": []
    }

    for i in range(len(stats)):
        results = {
            "prompt": prompts[i].base_structure(),
            "stats": stats[i]
        }
        data["results"].append(results)

    dir = "Results"
    if not os.path.exists(dir):
        os.makedirs(dir)

    date = datetime.now() - timedelta(hours=4)
    formatted_date = date.strftime('%Y%m%d-%H%M')
    path = f'{dir}/{formatted_date}.json'
    with open(path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False)

    return path

# Lee y muestra resultados de un archivo JSON
def read_results(path):
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    rows = []
    for result in data["results"]:
        for stat in result["stats"]:
            row = stat["MSE"]
            row["Prompt"] = json.dumps(result["prompt"])
            rows.append(row)

    df = pd.DataFrame(rows)
    col = df.pop('Prompt')
    df.insert(0, 'Prompt', col)
    col = df.pop('All')
    df.insert(1, 'All', col)

    def std(x): return np.std(x)
    df_mean = df.groupby('Prompt', as_index=False).mean().reset_index(drop=True)
    df_std = df.groupby('Prompt', as_index=False).agg(std).reset_index(drop=True)

    print("\nTabla Promedio MSE")
    display(df_mean)
    print("\nTabla Desviación estándar")
    display(df_std)

In [ ]:
x = experiment(df, prompts, repetitions=2, eval_function="map", eval_params=None, train_set_size=20, test_set_size=40, seed=42)

Entrenando Prompt 1 con Train Set 1
10-15-8-5-0-4-13-3-18-2-17-14-1-9-16-7-6-12-11-19-
Evaluando Prompt 1 con Test Set 1
4-33-36-38-3-31-35-21-37-27-23-13-0-32-1-2-20-24-17-6-11-5-8-34-30-39-28-7-9-29-22-12-10-19-16-26-25-18-

ERROR:openai_multi_client:Error processing Payload(endpoint='chats', data={'model': 'gpt-4o-mini', 'temperature': 0.1, 'n': 1, 'timeout': 10, 'messages': [{'role': 'user', 'content': '**Examples:**\nQ: ¿Cómo se podría implementar un historial de navegación web usando dos pilas? El historial debe permitir ir hacia atrás y adelante con las páginas previamente visitadas. Describa un algoritmo.\nIncorrect Answer: Usamos dos pilas para ir hacia adelante y hacia atrás en el historial.  (Score: 0)\n\nQ: ¿Cómo se busca un valor en un árbol rojo-negro? Explique el proceso paso a paso.\nIncorrect Answer: PAra buscar el valor en un árbol rojo-negro debemos pasar por nodos rojos y negros hasta encontrar el valor. (Score: 0)\n\nQ: ¿Por qué el acceso a un elemento específico en un arreglo es O(1), es decir, no depende de la cantidad de datos?\nIncorrect Answer: El acceso es O(1) por que toma un tiempo constante y no depende de la cantidad de datos. (Score: 0)\n\nQ: ¿Cuando se recomienda utilizar arr

15-14-
Entrenando Prompt 1 con Train Set 2
15-6-11-2-0-16-17-4-1-3-19-8-10-18-9-13-14-

ERROR:openai_multi_client:Error processing Payload(endpoint='chats', data={'model': 'gpt-4o-mini', 'temperature': 0.1, 'n': 1, 'timeout': 10, 'messages': [{'role': 'user', 'content': '**Examples:**\nQ: ¿Cómo se podría implementar un historial de navegación web usando dos pilas? El historial debe permitir ir hacia atrás y adelante con las páginas previamente visitadas. Describa un algoritmo.\nIncorrect Answer: Usamos dos pilas para ir hacia adelante y hacia atrás en el historial.  (Score: 0)\n\nQ: ¿Cómo se busca un valor en un árbol rojo-negro? Explique el proceso paso a paso.\nIncorrect Answer: PAra buscar el valor en un árbol rojo-negro debemos pasar por nodos rojos y negros hasta encontrar el valor. (Score: 0)\n\nQ: ¿Por qué el acceso a un elemento específico en un arreglo es O(1), es decir, no depende de la cantidad de datos?\nIncorrect Answer: El acceso es O(1) por que toma un tiempo constante y no depende de la cantidad de datos. (Score: 0)\n\nQ: ¿Cuando se recomienda utilizar arr

7-12-5-
Evaluando Prompt 1 con Test Set 2
0-7-34-2-16-22-35-36-8-3-5-12-30-19-23-26-1-15-18-39-38-13-31-4-37-17-25-9-21-11-33-24-10-29-28-27-20-6-14-32-

Tabla Promedio MSE


,Prompt,All,C1-OscarBadAnswers20,C2-Nan,C2-Sample100,C3-Sample100
0,"{""examples"": ""examples_1.txt"", ""context"": ""knowledge_1.txt"", ""question"": ""question_1.txt"", ""answer"": ""answer_1.txt"", ""instructions"": {""analysis"": ""analysis_1.txt"", ""feedback"": ""feedback_1.txt"", ""score"": ""score_1.txt""}}",0.87596,1.512279,0.73201,1.111855,0.64353



Tabla Desviación estándar


,Prompt,All,C1-OscarBadAnswers20,C2-Nan,C2-Sample100,C3-Sample100
0,"{""examples"": ""examples_1.txt"", ""context"": ""knowledge_1.txt"", ""question"": ""question_1.txt"", ""answer"": ""answer_1.txt"", ""instructions"": {""analysis"": ""analysis_1.txt"", ""feedback"": ""feedback_1.txt"", ""score"": ""score_1.txt""}}",0.095144,0.667108,0.255411,0.14933,0.358255


In [ ]:
read_results("Results/20240806-2347.json")


Tabla Promedio MSE


,Prompt,All,C1-OscarBadAnswers20,C2-Nan,C2-Sample100,C3-Sample100
0,"{""examples"": ""examples_1.txt"", ""context"": ""knowledge_1.txt"", ""question"": ""question_1.txt"", ""answer"": ""answer_1.txt"", ""instructions"": {""analysis"": ""analysis_1.txt"", ""feedback"": ""feedback_1.txt"", ""score"": ""score_1.txt""}}",0.87596,1.512279,0.73201,1.111855,0.64353



Tabla Desviación estándar


,Prompt,All,C1-OscarBadAnswers20,C2-Nan,C2-Sample100,C3-Sample100
0,"{""examples"": ""examples_1.txt"", ""context"": ""knowledge_1.txt"", ""question"": ""question_1.txt"", ""answer"": ""answer_1.txt"", ""instructions"": {""analysis"": ""analysis_1.txt"", ""feedback"": ""feedback_1.txt"", ""score"": ""score_1.txt""}}",0.095144,0.667108,0.255411,0.14933,0.358255
